In [1]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (8.0, 10.0)
import cv2
from pycocotools import mask
import pandas as pd

In [6]:
!pwd
!cd demo


/mmdetection


In [7]:
!pwd

/mmdetection


In [2]:
dataDir='../data/coco_datasets/datasets/merimen_coco/19_02_2022'
dataType='total'
annFile='{}/scratch/annotations/{}.json'.format(dataDir,dataType)

coco=COCO(annFile)


loading annotations into memory...


FileNotFoundError: [Errno 2] No such file or directory: '../data/coco_datasets/datasets/merimen_coco/19_02_2022/scratch/annotations/total.json'

In [ ]:
print(len(coco.anns), len(coco.imgs))

In [ ]:
# display COCO categories and supercategories
cats = coco.loadCats(coco.getCatIds())
print((cats))
nms=[cat['name'] for cat in cats]
print(nms)
print('COCO categories: \n{}\n'.format(' '.join(nms)))

nms = set([cat['supercategory'] for cat in cats])
print('COCO supercategories: \n{}'.format(' '.join(nms)))

In [ ]:
# get all images containing given categories, select one at random
catIds = coco.getCatIds(catNms=['scratch']);
imgIds = coco.getImgIds(catIds=catIds );
imgIds = coco.getImgIds(imgIds = [5519])
img = coco.loadImgs(imgIds[np.random.randint(0,len(imgIds))])[0]
print(img)

In [ ]:
I = io.imread('%s/data/coco_datasets/datasets/merimen_coco/19_02_2022/scratch/images/%s'%(dataDir,img['file_name']))
# use url to load image
# I = io.imread(img['scalabel_url'])
plt.axis('off')
plt.imshow(I)
plt.show()

In [ ]:
# load and display instance annotations
plt.imshow(I); plt.axis('off')
annIds = coco.getAnnIds(imgIds=img['id'], catIds=catIds, iscrowd=None)
anns = coco.loadAnns(annIds)
coco.showAnns(anns)

In [ ]:
coco.areas_seg = {}
coco.areas_bbox = {}
coco.mask_box_ratio = {}
coco.mask_image_ratio = {}
coco.box_image_ratio = {}

def compute_area(seg):
    seg = np.array(seg).reshape(-1,2).astype(np.int32)
    area = cv2.contourArea(seg)
    if area == 0:
        area = 1e-5
    return area

for ann in coco.dataset['annotations']:
    if ann['image_id'] == 5519:
        print(compute_area(ann['segmentation']))    

for ann in coco.dataset['annotations']:
    coco.areas_seg[ann['id']] = compute_area(ann['segmentation'])
    coco.areas_bbox[ann['id']] = ann['bbox'][2] * ann['bbox'][3] if ann['bbox'][2] * ann['bbox'][3] > 0 else 1e-5
    
    for img in coco.dataset['images']:
        if ann['image_id'] == img['id']:
            img_area = img['width'] * img['height']
    
    coco.mask_box_ratio[ann['id']] = coco.areas_seg[ann['id']]/coco.areas_bbox[ann['id']]
    coco.mask_image_ratio[ann['id']] = coco.areas_seg[ann['id']]/img_area
    coco.box_image_ratio[ann['id']] = coco.areas_bbox[ann['id']]/img_area



In [ ]:
print(coco.areas_seg[5519], coco.areas_bbox[5519])
print(len(coco.areas_seg), len(coco.areas_bbox))


In [ ]:
data_mask_box = [mask_box_ratio for mask_box_ratio in coco.mask_box_ratio.values()]
n , bins,_ =plt.hist(data_mask_box, label='mask_box', bins=50, edgecolor = 'w')
print(n)
print(bins)

In [ ]:
data_mask_img = [mask_img for mask_img in coco.mask_image_ratio.values()]
plt.hist([i for i in data_mask_img if i > 8.20158310e-08 and i <3.63267063e-02 ], 100, edgecolor = 'w')

In [ ]:
data_box_img = [box_img for box_img in coco.box_image_ratio]
plt.hist(data_box_img, 100, edgecolor='w')

In [ ]:
df_data = pd.DataFrame({'annotation_id': coco.areas_seg.keys(),'area_seg': coco.areas_seg.values(), 'area_bbox': coco.areas_bbox.values(), 
                        'mask_box_ratio': coco.mask_box_ratio.values(),'mask_image_ratio': coco.mask_image_ratio.values(),
                       'box_image_ratio': coco.box_image_ratio.values()})
df_data["area_bin"] = pd.cut(df_data["area_seg"],
                bins=[0, 32**2, 96**2, df_data["area_seg"].max()],
                labels=["Small", "Medium", "Large"])

In [ ]:
df_data

In [ ]:
df_data.to_csv('../data/coco_datasets/datasets/merimen_coco/19_02_2022/scratch/total_json_report.csv')


In [ ]:
df_data.hist()